---
# Introduction
- Vous pouvez (hmmm devez ?) ajouter des blocs de code comme des blocs d'explication.
- Pensez à sauvegarder régulièrement
- Il est posible de créer une copie locale de votre travail ```file / Download Notebook```. Il est alors possible de travailler en local après avoir installé un serveur Jupyter ainsi que les bibliothèques nécessaires.

Les principaux points abordés dans ce TP sont :
- La construction des ensembles à manipuler (apprentissage, validation, test)
- La visualisation des données
- Le choix des critères de séparation pour la création d'un arbre de décision
  + cas où la séparation peut s'effectuer sur un unique attribut
  + cas où la sépararation est linéaire, mais doit faire intervenir plusieurs attributs
- La construction de l'arbre
- Le jugement de la qualité de l'apprentissage
- Découverte de sklearn
- Introduction aux 'random forests'

---
---
# Avant l'apprentissage
---
---

---
# Préparation de l'environnement

## Charger les librairies 
- numpy
- pandas
- seaborn
- matplotlib.pyplot

In [1]:
# Charger les bibliothèques demandées
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.dummy import DummyClassifier

 ## Charger la base de données 'Student alcohol consumption' dans l'environnement
 - Commencer par incorporer la BD dans le kernel : *add data* dans la 'frame' de droite
 - Utiliser la commande pandas permettant de charger le fichier CSV student-mat des données (attention à l'indexage !)
 - N'oubliez pas de vérifier l'apparence du résultat en affichant les premières lignes
 
Rappel : les données se trouvent dans "../input/", dont on peut lister le contenu par :
 ```
import os
print(os.listdir("../input/"))
```
 

In [2]:
# Charger la BD 
student_data = pd.read_csv("../input/student-alcohol-consumption/student-mat.csv")
student_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/student-alcohol-consumption/student-mat.csv'

---
# Découverte et préparation de la base
- Lister les 3 premières lignes (préférer```display```  à ```print```)

In [ ]:
# Réaliser les premiers affichages
display(student_data.head(3))

- Afficher le nombre de lignes ainsi que le nombre de colonnes

In [ ]:
display(student_data.shape)

- Afficher les types des attributs + nombre de données manquantes par attribut

In [ ]:
data_types = student_data.dtypes
print("les attributs par type:")
display(data_types)

data_missing = student_data.isna().sum()
print("Le nombre de données manquantes par type:")
display(data_missing)

- Afficher un résumé statistique simple de cette base

In [ ]:
display(student_data.describe())

- Pour chaque attribut non continu, lister les valeurs possibles

In [ ]:
def list_val_attribut_non_continu(df):
    non_con = df.dtypes == 'object'
    for k,v in non_con.items():
        if v == True:
            display(k,df[k].unique())
            
list_val_attribut_non_continu(student_data)

- Remplacer les colonnes G1, G2, G3 par une nouvelle colonne G, valant la moyenne de G1, G2, G3

In [ ]:
student_data['G'] = student_data[['G1', 'G2', 'G3']].mean(axis=1)
new_st_data = student_data.drop(columns=['G1','G2','G3'])

- Remplacer les valeurs de G en suivant la règle suivante :
  - $x <8$ --> "A"
  - $8 \leq x<10$ --> "B"
  - $10 \leq x <13$ --> "C"
  - $13 \leq x<16$ --> "D"
  - $ x \geq 16$ --> "E"

In [ ]:
def replace_classe(col):
    if (col['G'] < 8):
        classe = 'A'
    elif (col['G'] >= 8 and col['G'] < 10):
        classe = 'B'
    elif (col['G'] >= 10 and col['G'] < 13):
        classe = 'C'
    elif (col['G'] >= 13 and col['G'] < 16):
        classe = 'D'
    else :
        classe = 'E'
    return classe

new_st_data['G'] = new_st_data.apply(replace_classe,axis=1)

- Compter le nombre de ligne pour chacune des valuations possibles de l'attribut "G". Dans la suite, le but du TP sera de prédir en fonction es autres attributs la valeur de l'attribut G.

In [ ]:
display(new_st_data['G'].value_counts())

- Calculer l'effectif de chacune des classes

In [ ]:
cat_cols = new_st_data.select_dtypes(include=object).columns.tolist()
(pd.DataFrame(new_st_data[cat_cols].melt(var_name='column', value_name='value').value_counts())
 .rename(columns={0: 'counts'}).sort_values(by=['column', 'counts']))

# Séparation de la base de données
- Lire le manuel de ```sklearn.model_selection.train_test_split```
  - Examiner en particulier l'option stratify, il faudra la mettre en oeuvre
  - Se poser la question de l'intérêt de random_state
- A l'aide de cette séparer la base de données en trois ensembles :
    - base d'apprentissage (50% des données) : train_X, train_Y
    - base de validation (25% des données) : validation_X, validation_Y
    - base de test (25% des données) : test_X, test_Y
- Rappeler la fonction de chacune de ces bases.

!!! Dans la suite, choisir avec attention sur quelle base les opérations doivent être réalisées !!!

In [ ]:
Y = new_st_data.G
col = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences']
X = new_st_data[col]

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.5, stratify=Y, random_state=1)
test_X, val_X, test_Y, val_Y = train_test_split(test_X, test_Y, train_size=0.5, stratify=test_Y, random_state=1)
print("train_x 50% c% correspondent à :",train_X.shape)
print("test_x 25% c% correspondent à :",test_X.shape)
print("val_x 25% c% correspondent à :",val_X.shape)

---
---
# Mise en place manuelle d'un arbre de décision
---
---

---
# Première découpe

## Pour chaque attribut
Représenter sur le même graphique la distribution des exemples en fonction de leur classe. On choisira un sns.histplot que l'on configurera au mieux.

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut school")
sns.histplot(data=new_st_data,x='G',hue='school', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut sex")
sns.histplot(data=new_st_data,x='G',hue='sex', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(12,3))
plt.title("Distribution des exemples sur l'attribut age")
sns.histplot(data=new_st_data,x='G',hue='age', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut address")
sns.histplot(data=new_st_data,x='G',hue='address', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut famsize")
sns.histplot(data=new_st_data,x='G',hue='famsize', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut Pstatus")
sns.histplot(data=new_st_data,x='G',hue='Pstatus', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution des exemples sur l'attribut Medu")
sns.histplot(data=new_st_data,x='G',hue='Medu', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution des exemples sur l'attribut Fedu")
sns.histplot(data=new_st_data,x='G',hue='Fedu', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution des exemples sur l'attribut Mjob")
sns.histplot(data=new_st_data,x='G',hue='Mjob', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution des exemples sur l'attribut Fjob")
sns.histplot(data=new_st_data,x='G',hue='Fjob', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(7,3))
plt.title("Distribution des exemples sur l'attribut reason")
sns.histplot(data=new_st_data,x='G',hue='reason', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut guardian")
sns.histplot(data=new_st_data,x='G',hue='guardian', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut traveltime")
sns.histplot(data=new_st_data,x='G',hue='traveltime', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut studytime")
sns.histplot(data=new_st_data,x='G',hue='studytime', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut failures")
sns.histplot(data=new_st_data,x='G',hue='failures', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut schoolsup")
sns.histplot(data=new_st_data,x='G',hue='schoolsup', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut famsup")
sns.histplot(data=new_st_data,x='G',hue='famsup', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut paid")
sns.histplot(data=new_st_data,x='G',hue='paid', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut activities")
sns.histplot(data=new_st_data,x='G',hue='activities', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut nursery")
sns.histplot(data=new_st_data,x='G',hue='nursery', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut higher")
sns.histplot(data=new_st_data,x='G',hue='higher', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut internet")
sns.histplot(data=new_st_data,x='G',hue='internet', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2))
plt.title("Distribution des exemples sur l'attribut romantic")
sns.histplot(data=new_st_data,x='G',hue='romantic', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut famrel")
sns.histplot(data=new_st_data,x='G',hue='famrel', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut freetime")
sns.histplot(data=new_st_data,x='G',hue='freetime', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut goout")
sns.histplot(data=new_st_data,x='G',hue='goout', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut Dalc")
sns.histplot(data=new_st_data,x='G',hue='Dalc', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,3))
plt.title("Distribution des exemples sur l'attribut Walc")
sns.histplot(data=new_st_data,x='G',hue='Walc', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(6,2.5))
plt.title("Distribution des exemples sur l'attribut health")
sns.histplot(data=new_st_data,x='G',hue='health', multiple='dodge', shrink=.6)

In [ ]:
plt.figure(figsize=(10,11))
plt.title("Distribution des exemples sur l'attribut absences")
sns.histplot(data=new_st_data,x='G',hue='absences', multiple='dodge', shrink=.8)

- Analyser les conséquences sur chacun des graphiques de réaliser un arbre de décision dont la racine utiliserait cet attribut afin de réaliser la première séparation.
Exemple : Si on séparait sur le critère de l'école, 
   - les deux branches seraient d'effectifs très déséquilibrées
   - on aurait peu d'informations nouvelle sur les 'GP' (la plus nombreuse)
   - sur les 'MS' la proportion de 'C' aurait augmenté par rapport à toutes les autres classes, et en particulier les proportions de B C E auraient particulièrement baissé.

- Choisir un attribut et une façon de scinder en deux cet attribut, par exemple :
Je choisi l'attribut 'activities freetime' car .... le côté gauche est défini par $\leq 4$ et le côté droit par $>4$

On choisit l'attribut **activities** et **paid**

- Créer deux nouveau dataframes 'd0' et 'd1' correspondant respectivement aux exemples du côté gauche et au exemples du côté droit.

In [ ]:
d0 = new_st_data.loc[new_st_data.activities == 'yes']
d1 = new_st_data.loc[new_st_data.activities == 'no']

---
# Niveau suivant de l'arbre
- Reprendre les graphiques en se limitant à la base 'd0', qui devra alors être découpée en 'd00' et 'd01'
- Reprendre les graphiques en se limitant à la base 'd1', qui devra alors être découpée en 'd10' et 'd11'

In [ ]:
#Reprise des graphiques en se limitant à la base d0

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut scool")
sns.histplot(data=d0,x='G',hue='school', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut sex")
sns.histplot(data=d0,x='G',hue='sex', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut address")
sns.histplot(data=d0,x='G',hue='address', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famsize")
sns.histplot(data=d0,x='G',hue='famsize', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Pstatus")
sns.histplot(data=d0,x='G',hue='Pstatus', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Medu")
sns.histplot(data=d0,x='G',hue='Medu', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Fedu")
sns.histplot(data=d0,x='G',hue='Fedu', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Mjob")
sns.histplot(data=d0,x='G',hue='Mjob', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Fjob")
sns.histplot(data=d0,x='G',hue='Fjob', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut reason")
sns.histplot(data=d0,x='G',hue='reason', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut guardian")
sns.histplot(data=d0,x='G',hue='guardian', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut traveltime")
sns.histplot(data=d0,x='G',hue='traveltime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut studytime")
sns.histplot(data=d0,x='G',hue='studytime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut failures")
sns.histplot(data=d0,x='G',hue='failures', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut schoolsup")
sns.histplot(data=d0,x='G',hue='schoolsup', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famsup")
sns.histplot(data=d0,x='G',hue='famsup', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut paid")
sns.histplot(data=d0,x='G',hue='paid', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut activities")
sns.histplot(data=d0,x='G',hue='activities', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut nursery")
sns.histplot(data=d0,x='G',hue='nursery', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut higher")
sns.histplot(data=d0 ,x='G',hue='higher', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut internet")
sns.histplot(data=d0 ,x='G',hue='internet', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut romantic")
sns.histplot(data=d0,x='G',hue='romantic', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famrel")
sns.histplot(data=d0,x='G',hue='famrel', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut freetime")
sns.histplot(data=d0,x='G',hue='freetime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut goout")
sns.histplot(data=d0,x='G',hue='goout', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Dalc")
sns.histplot(data=d0,x='G',hue='Dalc', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Walc")
sns.histplot(data=d0,x='G',hue='Walc', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut health")
sns.histplot(data=d0,x='G',hue='health', multiple='dodge', shrink=.6)

plt.figure(figsize=(11,11))
plt.title("Distribution des exemples sur l'attribut absences")
sns.histplot(data=d0,x='G',hue='absences', multiple='dodge', shrink=.6)

In [ ]:
d00 = d0.loc[d0.paid == 'no']
d01 = d0.loc[d0.paid == 'yes']

In [ ]:
#Reprise des graphiques en se limitant à la base d1

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut scool")
sns.histplot(data=d1,x='G',hue='school', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut sex")
sns.histplot(data=d1,x='G',hue='sex', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut address")
sns.histplot(data=d1,x='G',hue='address', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famsize")
sns.histplot(data=d1,x='G',hue='famsize', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Pstatus")
sns.histplot(data=d1,x='G',hue='Pstatus', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Medu")
sns.histplot(data=d1,x='G',hue='Medu', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Fedu")
sns.histplot(data=d1,x='G',hue='Fedu', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Mjob")
sns.histplot(data=d1,x='G',hue='Mjob', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Fjob")
sns.histplot(data=d1,x='G',hue='Fjob', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut reason")
sns.histplot(data=d1,x='G',hue='reason', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut guardian")
sns.histplot(data=d1,x='G',hue='guardian', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut traveltime")
sns.histplot(data=d1,x='G',hue='traveltime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut studytime")
sns.histplot(data=d1,x='G',hue='studytime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut failures")
sns.histplot(data=d1,x='G',hue='failures', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut schoolsup")
sns.histplot(data=d1,x='G',hue='schoolsup', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famsup")
sns.histplot(data=d1,x='G',hue='famsup', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut paid")
sns.histplot(data=d1,x='G',hue='paid', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut activities")
sns.histplot(data=d1,x='G',hue='activities', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut nursery")
sns.histplot(data=d1,x='G',hue='nursery', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut higher")
sns.histplot(data=d1 ,x='G',hue='higher', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut internet")
sns.histplot(data=d1 ,x='G',hue='internet', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut romantic")
sns.histplot(data=d1,x='G',hue='romantic', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut famrel")
sns.histplot(data=d1,x='G',hue='famrel', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut freetime")
sns.histplot(data=d1,x='G',hue='freetime', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut goout")
sns.histplot(data=d1,x='G',hue='goout', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Dalc")
sns.histplot(data=d1,x='G',hue='Dalc', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut Walc")
sns.histplot(data=d1,x='G',hue='Walc', multiple='dodge', shrink=.6)

plt.figure(figsize=(8,3))
plt.title("Distribution des exemples sur l'attribut health")
sns.histplot(data=d1,x='G',hue='health', multiple='dodge', shrink=.6)

plt.figure(figsize=(11,11))
plt.title("Distribution des exemples sur l'attribut absences")
sns.histplot(data=d1,x='G',hue='absences', multiple='dodge', shrink=.6)

In [ ]:
d10 = d1.loc[d1.paid == 'yes']
d11 = d1.loc[d1.paid == 'no']

---
# Fonction de découpe
- Ecrire une fonction qui attribue à une ligne du dataframe 'train' l'ensemble auquel cet élément est assigné, parmi 'd00, d01, d10, d11' (ce sont les feuilles de notre arbdre de décision)
- Déterminer la classe la plus fréquente dans chacun des ensembles 'd00, d01, d10, d11'
- Ecrire une fonction qui associe à une ligne du dataframe la classe majoritaire de l'ensemble auquel il est affecté

In [ ]:
#fonction permettant d'attribuer à une ligne du DataFrame l'ensemble auquel il est assigné
def ensemble_assigne(df,ligne):
    col = df.iloc[ligne,:]
    if(col['activities'] == 'yes'):
        if(col['paid'] == 'no'):
            ensemble = 'd00'
        elif(col['paid'] == 'yes'):
            ensemble = 'd01'
        else:
            ensemble = 'Aucun ensemble trouvé'
            
    elif(col['activities'] == 'no'):
        if(col['paid'] == 'yes'):
            ensemble = 'd10'
        elif(col['paid'] == 'no'):
            ensemble = 'd11'
        else:
            ensemble = 'Aucun ensemble trouvé'
    return ensemble
        

#fonction retournant la classe la plus féquente
def classe_plus_frequente(ensemble):
    classe = ensemble['G'].value_counts().idxmax()
    return classe

print("la classe la plus frequente pour d00 est : " + classe_plus_frequente(d00))
print("la classe la plus frequente pour d01 est : " + classe_plus_frequente(d01))
print("la classe la plus frequente pour d10 est : " + classe_plus_frequente(d10))
print("la classe la plus frequente pour d11 est : " + classe_plus_frequente(d11))

#Classe majoritaire associé à une ligne du Dataframe
def associe_classe(df,subsets,ligne):
    ensemble = ensemble_assigne(df,ligne)
    ensemble = subsets[ensemble]
    classe = classe_plus_frequente(ensemble)
    print("la classe majoritaire pour cette ligne est: " + classe)

subsets = {'d00': d00, 'd01': d01, 'd10': d10, 'd11': d11}

---
# Qualité de l'arbre
- Sachant que  nous allons essayer d'autres façons de réaliser l'apprentissage, sur quel ensemble doit-on réaliser la mesure de la qualité de l'arbre ainsi créé ?
- On choisit comme mesure de qualité le taux de bonnes prédictions (accuracy), le calculer.
- représenter la matrice de confusion

On mesurera la qualité de notre arbre sur l'ensemble **test_X**

---
# Arbre de décision réalisés par sklearn
---
- importer le module ```tree``` de ```sklearn```
- Etudier la documentation de ```DecisionTreeClassifier```, en particulier la partie **Tips on practical use**
- Construire un classifieur utilisant l'indice de Gini

Attention : actuellement, les arbres de décision de sklearn ne supportent que les données numériques, il  faudra donc réaliser un prétraitement sur les données afin de transformer les attributs catégoriels en attributs numériques.

In [ ]:
from sklearn import tree

#fonction permettant de convertir les données categorielles en numérique de mes données
def encode_data(df,col,data):
    for d in data:
        df[col] = df[col].replace(d[0],d[1])
        
def encoder_data(df):
    encode_data(df,'school',[('GP',0),('MS',1)])
    encode_data(df,'sex',[('F',0),('M',1)])
    encode_data(df,'address',[('U',0),('R',1)])
    encode_data(df,'famsize',[('LE3',0),('GT3',1)])
    encode_data(df,'Pstatus',[('A',0),('T',1)])
    
    encode_data(df,'Mjob',[('at_home',0),('health',1),('other',2),('services',3),('teacher',4)])
    encode_data(df,'Fjob',[('teacher',0),('other',1),('services',2),('health',3),('at_home',4)])
    encode_data(df,'reason',[('course',0),('other',1),('home',2),('reputation',3)])
    encode_data(df,'guardian',[('mother',0),('father',1),('other',2)])
    encode_data(df,'schoolsup',[('yes',0),('no',1)])
    encode_data(df,'famsup',[('yes',0),('no',1)])
    encode_data(df,'paid',[('no',0),('yes',1)])
    encode_data(df,'activities',[('no',0),('yes',1)])
    encode_data(df,'nursery',[('no',0),('yes',1)])
    encode_data(df,'higher',[('yes',0),('no',1)])
    encode_data(df,'internet',[('yes',0),('no',1)])
    encode_data(df,'romantic',[('no',0),('yes',1)])
    

encoder_data(train_X)
encoder_data(test_X)
encoder_data(val_X)


dtc = tree.DecisionTreeClassifier(criterion='gini')
dtc.fit(train_X, train_Y)

# Qualité

- calculer la matrice de confusion
- calculer les scores usuels de ce classifieur
- à l'aide de la fonction ```plot_confusion_matrix``` de sklearn, représenter graphiquement la matrice de confusion
- Comparer les scores à ceux obtenus par un classifieur 'bidon' (sklearn.dummy) (à quoi cela sert-il ?)

Rq : il reste un bug dans dummy, si vous obtenez une erreur de type 'no argmax on list', un contournement de ce problème peut être obtenu en reformattant les entrées du classifieur par 'check_X_y'


In [ ]:
predic = dtc.predict(train_X)
mc = metrics.confusion_matrix(train_Y,predic)
print("la matrice de confusion est la suivante :", mc)

#Calcul du score
score = metrics.accuracy_score(train_Y,predic)
print("le score du modèle est :", score)

##Affichage de la matrice de confusion
display_mc = metrics.plot_confusion_matrix(dtc, train_X, train_Y, display_labels = train_Y.unique(), cmap = plt.cm.Reds,
normalize = None)
display_mc.ax_.set_title("Matrice de confusion")


---
# Comparaison avec un classifieur primitif
Entrainer un 'dummy' classifieur (```sklearn.dummy.DummyClassifier```), et comparer les mesures du classifieur précédent à ce nouveau classifieur

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_X, train_Y)
print("le score du modèle dummy est:", dummy_clf.score(train_X, train_Y))

---
# Représenter un arbre de décision
- Importer le module graphviz
- utiliser la fonction de ```tree.export_graphviz``` puis ```graphviz.Source``` afin de réaliser une belle représentation graphique de l'arbre de décision

In [ ]:
import graphviz
graph_data = tree.export_graphviz(dtc, feature_names=train_X.columns,
                                  class_names=train_Y.unique(), filled = True, special_characters = True)
graph = graphviz.Source(graph_data)
display(graph)

---
# Construction du 'meilleur' arbre de décision
- faire varier les paramètres de construction de l'arbre de décision (bien mettre en pratique les 'Tips')
- choisir l'arbre le 'meilleur' sur l'ensemble de test  

N'oubliez pas le principe du ** rasoir d'Ockham ** pour effectuer votre choix !!!!!

---
# Random Forest
---
- Rappeler le principe des forets d'arbres décisionnelles, statquest est votre ami ! 
    - https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
    - https://www.youtube.com/watch?v=nyxTdL_4Q-Q

- Remplacer le classifieur par arbre de décision par un classifieur par une forêt d'arbres décisionnels

In [ ]:
#Creation du random forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100, criterion='gini')
rfc.fit(train_X, train_Y)


---
# Livraison
---
Faire les tests finaux afin de connaître la qualité de votre apprentissage, en reprenant les différentes mesures, mais cette fois sur l'ensemble de test.

In [ ]:
dtc = tree.DecisionTreeClassifier(criterion='gini')
dtc.fit(val_X, val_Y)

predic = dtc.predict(val_X)
mc = metrics.confusion_matrix(val_Y,predic)
print("la matrice de confusion est la suivante :", mc)

#Calcul du score
score = metrics.accuracy_score(val_Y,predic)
print("le score du modèle est :", score)

##Affichage de la matrice de confusion
display_mc = metrics.plot_confusion_matrix(dtc, val_X, val_Y, display_labels = val_Y.unique(), cmap = plt.cm.Reds,
normalize = None)
display_mc.ax_.set_title("Matrice de confusion")

graph_data = tree.export_graphviz(dtc, out_file=None, feature_names=val_X.columns,
                                  class_names=val_Y.unique(), filled = True, special_characters = True)
graph = graphviz.Source(graph_data)
display(graph)

In [ ]:
dtc = tree.DecisionTreeClassifier(criterion='gini')
dtc.fit(test_X, test_Y)

predic = dtc.predict(test_X)
mc = metrics.confusion_matrix(test_Y,predic)
print("la matrice de confusion est la suivante :", mc)

#Calcul du score
score = metrics.accuracy_score(test_Y,predic)
print("le score du modèle est :", score)

##Affichage de la matrice de confusion
display_mc = metrics.plot_confusion_matrix(dtc, test_X, test_Y, display_labels = test_Y.unique(), cmap = plt.cm.Reds,
normalize = None)
display_mc.ax_.set_title("Matrice de confusion")

graph_data = tree.export_graphviz(dtc, out_file=None, feature_names=test_X.columns,
                                  class_names=test_Y.unique(), filled = True, special_characters = True)
graph = graphviz.Source(graph_data)
display(graph)

# Merci d'être allé jusqu'à la fin du TP, j'espère que ce travail vous a aidé à approfondir votre compréhension du cours d'apprentissage artificiel.